<a href="https://placekey.io/">
         <img src="https://assets.website-files.com/5f08ccbb93b299154d34ef7f/5f1e9c2bdd643e0d63967ca9_logo-placekey-black.svg" alt="Placekey" ALIGN="left" width=600>
      </a>



# Quick Start Guide For Batch Process [Placekey](https://placekey.io/) Matching With Python
--------------
**[Ryan Fox Squire](https://www.linkedin.com/in/ryanfoxsquire/) | Senior Data Scientist, [SafeGraph](https://safegraph.com/)**

 <img src="https://global-uploads.webflow.com/5baafc2653bd67278f206724/5be267a03f7813daf821b31e_safegraph-logo-hidpi%403x-p-500.png" alt="SafeGraph" ALIGN="center" width=100>


Sep 2020, updated Dec 2020

--------------
[*Open this notebook in Google CoLab*](https://colab.research.google.com/github/Placekey/placekey-notebooks/blob/main/notebooks/Adding_Placekey_to_your_POI_dataset_using_python_and_the_Placekey_API.ipynb)

[*View this notebook on Github*](https://github.com/Placekey/placekey-notebooks/blob/main/notebooks/Adding_Placekey_to_your_POI_dataset_using_python_and_the_Placekey_API.ipynb)



## Summary
* [Placekey](https://placekey.io/) is geospatially-aware universal place identifier useful for joining together disparate geospatial, address, and point-of-interest datasets. 
* Placekey is an open and free to use.
* Here we demonstrate how to obtain Placekeys for point-of-interest or address-based data using the [placekey-py](https://github.com/Placekey/placekey-py) python library and Placekey API. 


#### What do you need? 
* Your Placekey API key --> [Register Here](https://dev.placekey.io/default/register) 
* Your dataset containing points-of-interest, addresses, and/or geospatial coordinates for which you want to assign a universal place identifier (Placekey)


#### Have questions or ideas? 
 * Join thousands of geospatial data scientists in the [Placekey Slack Community](https://www.placekey.io/community) to get help and   share ideas.
 * Follow [Placekey on Github](https://github.com/Placekey)
 * Tweet [@placekeyio](https://twitter.com/placekeyio?lang=en)

In [42]:
!pip install placekey --upgrade

import pandas as pd
import numpy as np
import json
from placekey.api import PlacekeyAPI

## Set Your API Key

Get your Placekey API instantly here: [Register for Placekey API](https://dev.placekey.io/default/register) 


In [43]:
placekey_api_key = "ENTER YOUR API KEY HERE" # fill this in with your personal API key (do not share publicly)
# placekey_api_key = pd.read_csv("/Users/ryansquire/Documents/datasets/demo_keys/squire_placekey_api_key").columns[0] # for original author debugging

pk_api = PlacekeyAPI(placekey_api_key)

## Read in Data
* Define `data_path` and `data_filename`
* Mind your datatypes (do not drop leading zeros for postal code)


In [44]:
data_path = "/Users/ryansquire/Documents/datasets/CostcoWholesaleCorp-Sept-2020-CORE_POI/" 
data_filename = "my_places_data_missing"
file_suffix = ".csv"
full_path = data_path + data_filename + file_suffix
print(full_path)

/Users/ryansquire/Documents/datasets/CostcoWholesaleCorp-Sept-2020-CORE_POI/my_places_data_missing.csv


In [45]:
my_dtypes = {'internal_place_id':str, 
             'store_name' : str,
             'street_address':str, 
             'city': str, 
             'state': str, 
             'zip_code': str, # This is important so that you do not drop leading zeros
             'iso_country_code':str,
             'lat' : float,
             'lon' : float}
orig_df = pd.read_csv(full_path, dtype=my_dtypes)
print(orig_df.shape)
orig_df.head()

(545, 8)


,internal_place_id,store_name,lat,lon,full_address,city,state,zip_code
0,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,33.804504,-118.039593,5401 Katella Ave,Cypress,CA,90720
1,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,NaN,NaN,25900 SW Heather Pl,Wilsonville,OR,NaN
2,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,47.445371,-122.248623,400 Costco Dr Ste 150,Tukwila,WA,98188
3,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,41.844777,-87.806473,2500 Harlem Ave,North Riverside,IL,60546
4,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,38.508769,-90.338324,4200 Rusty Rd,Concord,MO,63128


## Prep Data Column Names for Placekey API

https://docs.placekey.io/

Make sure to list every column useful for the Placekey API in `column_map` even if you do not need to rename it (see `city` in the example).

In [23]:
query_id_col = "internal_place_id" # this column in your data should be unique for every row
column_map = {query_id_col: "query_id",
              "store_name" : "location_name",
              "full_address" : "street_address",
              "city": "city",
              "state": "region",
              "zip_code": "postal_code",
              "lat": "latitude",
              "lon": "longitude"
             }

df_for_api = orig_df.rename(columns=column_map)
cols = list(column_map.values())
df_for_api = df_for_api[cols]

# add missing hard-coded columns
df_for_api['iso_country_code'] = 'US'
df_for_api.head()

,query_id,location_name,street_address,city,region,postal_code,latitude,longitude,iso_country_code
0,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,5401 Katella Ave,Cypress,CA,90720,33.804504,-118.039593,US
1,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,25900 SW Heather Pl,Wilsonville,OR,NaN,NaN,NaN,US
2,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,400 Costco Dr Ste 150,Tukwila,WA,98188,47.445371,-122.248623,US
3,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,2500 Harlem Ave,North Riverside,IL,60546,41.844777,-87.806473,US
4,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,4200 Rusty Rd,Concord,MO,63128,38.508769,-90.338324,US


## Data Cleaning Best Practices

Gotcha's to check for:
* ""
* " "
* "null"/"Null"
* "None"
* Missing / Nan
* Anything else weird in your data? 
* Convert all of these to `np.nan` so that they are handled correctly by `json` library

In [46]:
df_clean = df_for_api.copy()
possible_bad_values = ["", " ", "null", "Null", "None", "nan", "Nan"] # Any other dirty data you need to clean up? 
for bad_value in possible_bad_values:
  df_clean = df_clean.replace(to_replace=bad_value, value=np.nan)

print("FYI data missing from at least 1 column in the following number of rows:")
print(df_clean.shape[0] - df_clean.dropna().shape[0])
print("Some examples of rows with missing data")
df_clean[df_clean.isnull().any(axis=1)].head()

FYI data missing from at least 1 column in the following number of rows:
18
Some examples of rows with missing data


,query_id,location_name,street_address,city,region,postal_code,latitude,longitude,iso_country_code
1,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,25900 SW Heather Pl,Wilsonville,OR,NaN,NaN,NaN,US
5,sg:03cb6aeb0e024bd6bf71626e30b201a0,Costco Wholesale Corp.,5901 Redwood Dr,Rohnert Park,CA,94928,NaN,NaN,US
10,sg:377f1c2a3c3e4b0885b8addcb4f70997,Costco Wholesale Corp.,1501 W Kelly Ave,Pharr,TX,78577,NaN,NaN,US
12,sg:98ccf43c4a934f30a943a9564ce4cf0e,Costco Wholesale Corp.,24008 Snohomish Woodinville Rd,Maltby,WA,NaN,47.779382,-122.148732,US
16,sg:be95a4498ee340db9b5eeb3130124eab,Costco Wholesale Corp.,1415 N Arizona Ave,Gilbert,AZ,NaN,33.376033,-111.840291,US


In this example example dataset some rows are missing `postal_code`, `latitude`/`longitude`, and even the `street_address`. 

The Placekey API is flexible and powerful and can often handle incomplete data. We will send the API the fields we have and see what comes back. 

### Prepare Data Structure For Batch API 

https://docs.placekey.io/#5778b1b7-ce57-4cbf-a74e-bf4e2ce5a9ec

Advanced: If you want to include options like [`strict_address_matching`](https://docs.placekey.io/#77042b13-ad19-423b-b117-c1ec819ae2f9) for more conservative results, edit the function `prepare_batches_for_API`. 

In [47]:
data_jsoned = json.loads(df_clean.to_json(orient="records"))
print("number of records: ", len(data_jsoned))
print("example record:")
data_jsoned[0]

number of records:  545
example record:


{'query_id': 'sg:22a7df9c14c148bb969171d93e0950c3',
 'location_name': 'Costco Wholesale Corp.',
 'street_address': '5401 Katella Ave',
 'city': 'Cypress',
 'region': 'CA',
 'postal_code': '90720',
 'latitude': 33.804504,
 'longitude': -118.039593,
 'iso_country_code': 'US'}

## Request Placekeys From Placekey API and View Results



##### Placekey for a single place

You can use the function [`lookup_placekey()`](https://placekey.github.io/placekey-py/placekey.html#placekey.api.PlacekeyAPI.lookup_batch) to get a Placekey for a single place. 

In [56]:
single_place_example = data_jsoned[0]
print("input: \n",single_place_example)
print("\nresult: \n",pk_api.lookup_placekey(**single_place_example))

input: 
 {'query_id': 'sg:22a7df9c14c148bb969171d93e0950c3', 'location_name': 'Costco Wholesale Corp.', 'street_address': '5401 Katella Ave', 'city': 'Cypress', 'region': 'CA', 'postal_code': '90720', 'latitude': 33.804504, 'longitude': -118.039593, 'iso_country_code': 'US'}

result: 
 {'query_id': 'sg:22a7df9c14c148bb969171d93e0950c3', 'placekey': 'zzw-224@5z6-3p2-v9f'}


##### Placekeys for many places (batch)
You can use the function [`lookup_placekeys()`](https://placekey.github.io/placekey-py/placekey.html#placekey.api.PlacekeyAPI.lookup_placekeys) (plural) to get placekeys for many places. 

In [67]:
%%time
responses = pk_api.lookup_placekeys(data_jsoned, 
                                    strict_address_match=False,
                                    strict_name_match=False, 
                                    verbose=True)
print("Example:", responses[0])

2020-12-07 11:22:36,268	INFO	Processed 545 items
2020-12-07 11:22:36,269	INFO	Done


Example: {'query_id': 'sg:22a7df9c14c148bb969171d93e0950c3', 'placekey': 'zzw-224@5z6-3p2-v9f'}
CPU times: user 107 ms, sys: 6.77 ms, total: 114 ms
Wall time: 4.6 s


##### Advanced

The functions `lookup_placekey()` and `lookup_placekeys()` have several optional parameters. For more conservative results, consider setting the parameters `strict_address_match` and `strict_name_match` to `True` (otherwise, defaults to `False`).



##### Verify valid results were received

In [68]:
def clean_api_responses(data_jsoned, responses):

    print("number of original records: ", len(data_jsoned))
    print("total individual queries returned:", len(responses))

    # filter out invalid responses
    responses_cleaned = [resp for resp in responses if 'query_id' in resp]
    print("total successful query responses:", len(responses_cleaned))
    return(responses_cleaned)

responses_cleaned = clean_api_responses(data_jsoned, responses)

number of original records:  545
total individual queries returned: 545
total successful query responses: 545


In [69]:
df_placekeys = pd.read_json(json.dumps(responses_cleaned), dtype={'query_id':str})
df_placekeys.head(10)

,query_id,placekey,error
0,sg:22a7df9c14c148bb969171d93e0950c3,zzw-224@5z6-3p2-v9f,NaN
1,sg:235e4067e4b04332b23e5a38448f6535,222-222@5xc-q57-xqz,NaN
2,sg:67591b2e593048a4bd4e09e027ba9caf,223-223@5x4-4sg-vfz,NaN
3,sg:b568fdaf8fd64651ad1f5931e48440fe,zzw-223@5pw-68m-4jv,NaN
4,sg:ccacecad3a01486a8873a05a1ad9db41,222-222@5pj-24r-7t9,NaN
5,sg:03cb6aeb0e024bd6bf71626e30b201a0,222-222@5vg-55k-p7q,NaN
6,sg:3381832b1ba14db09a140e9f13bb386a,zzw-222@5xd-qd6-ghq,NaN
7,sg:2f347f46f0a34017a061bdf8d7430d01,zzw-222@5qc-x2b-7nq,NaN
8,sg:20d033ca8fe44c31a02d7325ce069bd3,222-222@5vg-87t-psq,NaN
9,sg:57f9bfb8ae45495d97f5af97f228d143,222-222@63v-bky-cbk,NaN


## Add `placekey` Column Back to Original Dataset and Save Results

In [62]:
df_join_placekey = pd.merge(orig_df, df_placekeys, left_on=query_id_col, right_on="query_id", how='left')
final_cols = list(df_placekeys.columns) + list(orig_df.columns)
df_join_placekey = df_join_placekey[final_cols]
df_join_placekey.head()

,query_id,placekey,error,internal_place_id,store_name,lat,lon,full_address,city,state,zip_code
0,sg:22a7df9c14c148bb969171d93e0950c3,zzw-224@5z6-3p2-v9f,NaN,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,33.804504,-118.039593,5401 Katella Ave,Cypress,CA,90720
1,sg:235e4067e4b04332b23e5a38448f6535,222-222@5xc-q57-xqz,NaN,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,NaN,NaN,25900 SW Heather Pl,Wilsonville,OR,NaN
2,sg:67591b2e593048a4bd4e09e027ba9caf,223-223@5x4-4sg-vfz,NaN,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,47.445371,-122.248623,400 Costco Dr Ste 150,Tukwila,WA,98188
3,sg:b568fdaf8fd64651ad1f5931e48440fe,zzw-223@5pw-68m-4jv,NaN,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,41.844777,-87.806473,2500 Harlem Ave,North Riverside,IL,60546
4,sg:ccacecad3a01486a8873a05a1ad9db41,222-222@5pj-24r-7t9,NaN,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,38.508769,-90.338324,4200 Rusty Rd,Concord,MO,63128


In [63]:
print("Summary of results:")
total_recs = df_join_placekey.shape[0]
print("total records:", total_recs)
print("records with a placekey: {0} | {1:.2f}%".format(df_join_placekey[~df_join_placekey.placekey.isnull()].shape[0],  df_join_placekey[~df_join_placekey.placekey.isnull()].shape[0]*100/total_recs))      
print("records missing a placekey: {0} | {1:.2f}% [Missing placekeys may be due to exceeding API rate limit]".format(df_join_placekey[df_join_placekey.placekey.isnull()].shape[0], df_join_placekey[df_join_placekey.placekey.isnull()].shape[0]*100/total_recs))
print("records missing a query_id: {0} | {1:.2f}% [Invalid query inputs or missing result due to exceeded API rate limit]".format(df_join_placekey[df_join_placekey.query_id.isnull()].shape[0], df_join_placekey[df_join_placekey.query_id.isnull()].shape[0]*100/total_recs))

Summary of results:
total records: 545
records with a placekey: 542 | 99.45%
records missing a placekey: 3 | 0.55% [Missing placekeys may be due to exceeding API rate limit]
records missing a query_id: 0 | 0.00% [Invalid query inputs or missing result due to exceeded API rate limit]


In [64]:
output_cols = ['placekey'] + list(orig_df.columns)
output_df = df_join_placekey[output_cols]

write_path = data_path + data_filename + "_placekeys" + file_suffix
output_df.to_csv(write_path, index=False)
print("wrote to: ", write_path)
output_df.head(5)

wrote to:  /Users/ryansquire/Documents/datasets/CostcoWholesaleCorp-Sept-2020-CORE_POI/my_places_data_missing_placekeys.csv


,placekey,internal_place_id,store_name,lat,lon,full_address,city,state,zip_code
0,zzw-224@5z6-3p2-v9f,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,33.804504,-118.039593,5401 Katella Ave,Cypress,CA,90720
1,222-222@5xc-q57-xqz,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,NaN,NaN,25900 SW Heather Pl,Wilsonville,OR,NaN
2,223-223@5x4-4sg-vfz,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,47.445371,-122.248623,400 Costco Dr Ste 150,Tukwila,WA,98188
3,zzw-223@5pw-68m-4jv,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,41.844777,-87.806473,2500 Harlem Ave,North Riverside,IL,60546
4,222-222@5pj-24r-7t9,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,38.508769,-90.338324,4200 Rusty Rd,Concord,MO,63128


## Success!

#### Have questions or ideas? 
 * Join thousands of geospatial data scientists in the [Placekey Slack Community](https://www.placekey.io/community) to get help from Placekey experts and to share your ideas.
 * Follow [Placekey on Github](https://github.com/Placekey).
 * [@placekeyio](https://twitter.com/placekeyio?lang=en).

<br>
<br>
<br>
<br>

<a href="https://placekey.io/">
         <img src="https://assets.website-files.com/5f08ccbb93b299154d34ef7f/5f1e9c2bdd643e0d63967ca9_logo-placekey-black.svg" alt="Placekey" ALIGN="left" width=300>
      </a>

  